In [1]:
import warnings
from typing import Union, Tuple
from collections import defaultdict

import requests

from random import choice, shuffle

from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.api.queries.glif_api import GlifApi

import pickle
import os, random
import numpy as np
import matplotlib.pyplot as plt

ctc = CellTypesCache(manifest_file='../cell_types/manifest.json')

random.seed(0, version=2)



def obtain_spike_time_and_current_and_voltage(cell_id = 324257146):
    try:
        raw_data = ctc.get_ephys_data(cell_id)
        sweep_numbers = raw_data.get_sweep_numbers()
    except:
        print('can not download data...')
    data_set = []
    for sweep_id in sweep_numbers:
        sweep_data = {}
        #print('processing sweep with sweep id = '+ str(sweep_id))
        
        stimulus_name = raw_data.get_sweep_metadata(sweep_id)['aibs_stimulus_name'].decode()
        sweep_data['stimulus_name'] = stimulus_name
        
        sampling_rate = raw_data.get_sweep(sweep_id)['sampling_rate']
        sweep_data['sampling_rate'] = sampling_rate
        
        # start/stop indices that exclude the experimental test pulse (if applicable)
        index_range = raw_data.get_sweep(sweep_id)['index_range']
        
        I = 1E12*raw_data.get_sweep(sweep_id)['stimulus']
        V = 1E3*raw_data.get_sweep(sweep_id)['response']
        spike_times = raw_data.get_spike_times(sweep_id)
        
        #make sure index_range[1] is corrected for ramp:
        if stimulus_name == 'Ramp':
            max_I_idx = np.argmax(I) if np.argmax(I) > index_range[0] else None
            voltage_dies_at = np.max(np.nonzero(V))
            index_range_1 = min(index_range[1],voltage_dies_at)
            if max_I_idx is not None:
                index_range_1 = min(index_range_1,max_I_idx)

            index_range_0 = index_range[0]
            index_range = (index_range_0,index_range_1)
        sweep_data['index_range'] = index_range
        
        sweep_data['current'] = I
        sweep_data['spike_times'] = spike_times
        sweep_data['voltage'] = V
        
        data_set.append(sweep_data)


    with open('Calvin_raw_data/raw_data_'+str(cell_id)+'.pickle', 'wb') as handle:
        pickle.dump(data_set, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return data_set


ModuleNotFoundError: No module named 'allensdk'

In [ ]:
for cell_id in [x['id'] for x in ctc.get_cells(species=[CellTypesApi.MOUSE])]:
    if os.path.isfile('Calvin_raw_data/raw_data_'+str(cell_id)+'.pickle'):
        pass
    else: 
        try:
            data_set=obtain_spike_time_and_current_and_voltage(cell_id = cell_id)
        except:
            print('unprocessed cell: ' + str(cell_id))